# DATASET

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path="/content/drive/MyDrive/Colab Notebooks/MobileDEV/"
path_ner="ner.csv"
path_ner_dataset="ner_dataset.csv"

In [3]:
from IPython.display import clear_output 
!pip install sklearn-crfsuite
!pip install tabulate
!pip install nervaluate
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from tabulate import tabulate
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import itertools
from nervaluate import Evaluator
from tqdm.auto import tqdm
import re
clear_output()

tqdm.pandas()

In [4]:
print(f"Numpy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

Numpy version: 1.21.6
Pandas version: 1.3.5


Auxiliary functions

In [5]:
tag_name_pattern = re.compile(r"(B|I)-")

def map_seq(seq, dictionary):
    return [dictionary[elem] for elem in seq]

In [9]:
data1 = pd.read_csv(path+path_ner, encoding = "ISO-8859-1", index_col=0, error_bad_lines=False)
data2 = pd.read_csv(path+path_ner_dataset, encoding="latin1")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 281837: expected 25 fields, saw 34\n'


In [10]:
chrs = ["\x85", "\x94"]
lengths = data2["Word"].apply(lambda x: len(x.split()))
data2 = data2[lengths == 1]
data2 = data2[data2["Word"].apply(lambda x: x not in chrs)]
data2 = data2.reset_index(drop=True)
''' data2 = data2.fillna(method="ffill")
data2.head() '''
data2.head(50)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


In [11]:
first_words_inds = data2[data2["Sentence #"].notna()].index.to_list()
for i in tqdm(range(len(first_words_inds)-1)):
    data2["Sentence #"].loc[range(first_words_inds[i], first_words_inds[i+1])] = i
data2["Sentence #"].loc[range(first_words_inds[-1],data2["Sentence #"].shape[0])] = len(first_words_inds) - 1
data2 = data2.astype({"Sentence #": int, "Word": "string", "POS": "string", "Tag": "string"})

  0%|          | 0/47958 [00:00<?, ?it/s]

In [12]:
data2.head(50)

,Sentence #,Word,POS,Tag
0,0,Thousands,NNS,O
1,0,of,IN,O
2,0,demonstrators,NNS,O
3,0,have,VBP,O
4,0,marched,VBN,O
5,0,through,IN,O
6,0,London,NNP,B-geo
7,0,to,TO,O
8,0,protest,VB,O
9,0,the,DT,O


In [13]:
texts_df = data2[["Sentence #", "Word", "POS", "Tag"]].groupby(by="Sentence #").aggregate(lambda x: " ".join(x))
texts_df.columns = ["text", "pos seq", "tag seq"]

# HMM

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn_crfsuite.metrics import flat_classification_report

In [ ]:
class HMMTaggerTemplate():
    def __init__(self, states, observations):
        """Initialize HMM model with states and observations
        :param states: array with unique hidden states (list)
        :param observations: array with unique observations (list)
        """
        # add 'Unk' to handle unkown tokens
        self.states = states
        self.observations = [*observations, 'Unk']
        self.states_num = len(self.states)
        self.observations_num = len(self.observations)

        self.init_prob = np.zeros(shape=(1, self.states_num))
        self.transition_matrix = np.zeros(shape=(self.states_num,self.states_num))
        self.emission_matrix = np.zeros(shape=(self.states_num, self.observations_num))

        self.states_to_idx = {state:idx for idx, state in enumerate(self.states)}
        self.observations_to_idx = {obs:idx for idx, obs in enumerate(self.observations)}
        

    def fit(self, train_data):
        """Estimate initial probability vector, transition and emission matrices
        :param train_data: list of sentecnes where each sentence is represented by list of tuples (state, observation)
        
        Example: train_data = [[('O', 'Thousands'),
                                ('O', 'of'),
                                ('O', 'demonstrators'),
                                ('O', 'have'),
                                ('O', 'marched'),
                                ('O', 'through'),
                                ('B-geo', 'London')],
                               [('B-gpe', 'Iranian'),
                                ('O', 'officials'),
                                ('O', 'say'),
                                ('O', 'they'),
                                ('O', 'expect'),
                                ('O', 'to'),
                                ('O', 'get'),
                                ('O', 'access')]]
                                
        """
        self.emission_matrix += 1 # smoothing
        c_final = np.zeros(shape=(1, self.states_num))
        
        for example in train_data:
          first_state_ind = self.states_to_idx[example[0][0]]
          last_state_ind = self.states_to_idx[example[-1][0]]
          last_obs_ind = self.observations_to_idx[example[-1][1]]
          
          self.init_prob[0, first_state_ind] += 1
          c_final[0, last_state_ind] += 1

          for ind in range(len(example)-1):
            curr_state_ind = self.states_to_idx[example[ind][0]]
            curr_obs_ind = self.observations_to_idx[example[ind][1]]
            next_state_ind = self.states_to_idx[example[ind+1][0]]
            
            self.transition_matrix[next_state_ind, curr_state_ind] += 1
            self.emission_matrix[curr_state_ind, curr_obs_ind] += 1
             
          self.emission_matrix[last_state_ind, last_obs_ind] += 1
        
        self.init_prob = self.init_prob / np.sum(self.init_prob)
        self.transition_matrix = (self.transition_matrix / (np.sum(self.transition_matrix, axis=0))).T
        self.emission_matrix = self.emission_matrix / np.sum(self.emission_matrix, axis=1).reshape(-1, 1)
        #return self
    

    def __viterbi(self, obs_sequence_indices):
        """Decode incoming sequence of observations into the most propable sequence of hidden states using Viterbi algorithm
        : param obs_sequence_indices: list of observations indices
        :return: list of hidden states indices
        """
        tmp = [0]*self.states_num

        delta = [tmp[:]]  # Compute initial state probabilities
        for i in range(self.states_num):
          delta[0][i] = self.init_prob[0,i] * self.emission_matrix[i, obs_sequence_indices[0]]

        phi = [tmp[:]]

        for obs in obs_sequence_indices[1:]:  # For all observations except the inital one
          delta_t = tmp[:]
          phi_t = tmp[:]
          for j in range(self.states_num):   # Following formula 33 in Rabiner'89
            tdelta = tmp[:]
            tphimax = -1.0
            for i in range(self.states_num):
              tphi_tmp = delta[-1][i] * self.transition_matrix[i,j]
              if (tphi_tmp > tphimax):
                tphimax = tphi_tmp
                phi_t[j] = i
              tdelta[i] = tphi_tmp * self.emission_matrix[j, obs]
            delta_t[j] = max(tdelta)
          delta.append(delta_t)
          phi.append(phi_t)

        # Backtrack the path through the states  (Formula 34 in Rabiner'89)
        #
        tmax = -1.0
        for i in range(self.states_num):
          if (delta[-1][i] > tmax):
            tmax = delta[-1][i]
            state_seq = [i]  # Last state with maximum probability

        phi.reverse()  # Because we start from the end of the sequence
        for tphi in phi[:-1]:
          state_seq.append(tphi[state_seq[-1]])
        return reversed(state_seq)


    def predict(self, obser_seq):
        """Decode observable sequences using Viterbi algorithm
        :param obser_seq: list of sentences where each sentence is represented by list of observations
        :return: list of the most probable hidden states

        Example: obser_seq = [['The','military','says','the','blast'],
                              ['The','attack','prompted','Scandinavian','monitors','overseeing','Sri','Lanka']]
        """
             
        result = []
        
        for seq in tqdm(obser_seq):
          obser_inds_seq = [self.observations_to_idx[token] for token in seq]
          state_ind_seq = list(self.__viterbi(obser_inds_seq))
          state_seq = [self.states[state_ind] for state_ind in state_ind_seq]
          result.append(state_seq)
        return result

In [ ]:
texts = texts_df["text"].apply(lambda x: x.split())
tags = texts_df["tag seq"].apply(lambda x: x.split())
X_train, X_test, y_train, y_test = train_test_split(texts.to_numpy(), tags.to_numpy(), test_size=0.15, random_state=42)

In [ ]:
ziped_train = []
for pair in np.stack((X_train, y_train), axis=1):
    ziped_train.append(np.stack((pair[1], pair[0]), axis=1))

In [ ]:
states = data2["Tag"].unique().to_numpy()
observ = data2["Word"].unique().to_numpy()
hmm = HMMTaggerTemplate(states, observ) 

In [ ]:
hmm.fit(ziped_train)

In [ ]:
y_pred = hmm.predict(X_test)

In [ ]:
y_test_flat = list(itertools.chain.from_iterable(y_test))
y_pred_flat = list(itertools.chain.from_iterable(y_pred))

In [ ]:
print(classification_report(y_test_flat, y_pred_flat))

# CRF

In [ ]:
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn.model_selection import cross_val_predict, RandomizedSearchCV
from sklearn.metrics import make_scorer
import scipy.stats
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [ ]:
sentences = [[]]
cur_sent_ind = 0
for _, row in tqdm(data2.iterrows()):
    sent_ind, word, pos, tag = row.to_list()
    
    if sent_ind == cur_sent_ind:
        sentences[-1].append((word, pos, tag))
    else:
        sentences.append([(word, pos, tag)])
        cur_sent_ind = sent_ind

In [ ]:
featured_sentences = [sent2features(s) for s in tqdm(sentences)]
tags = [sent2labels(s) for s in tqdm(sentences)]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(featured_sentences, tags, test_size=0.2)

In [ ]:
train_set_size_for_random_search = 8000
inds = np.random.choice(range(len(X_train)), size=train_set_size_for_random_search)
X_train_random = [X_train[i] for i in inds]
y_train_random = [y_train[i] for i in inds]

In [ ]:
crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [ ]:
!pip install git+https://github.com/MeMartijn/updated-sklearn-crfsuite.git#egg=sklearn_crfsuite

In [ ]:
''' crf = CRF(algorithm='lbfgs',
          c1=0.06084,
          c2=0.03932,
          max_iterations=100,
          all_possible_transitions=False) '''
''' crf.fit(X_train, y_train) '''

try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass


In [ ]:
y_pred = crf.predict(X_test)

In [ ]:
y_test_flat = list(itertools.chain.from_iterable(y_test))
y_pred_flat = list(itertools.chain.from_iterable(y_pred))

In [ ]:
print(classification_report(y_test_flat, y_pred_flat))

# RNN

In [ ]:
''' data2 = data2.fillna(method="ffill") '''


In [15]:
data2.head(50)

,Sentence #,Word,POS,Tag
0,0,Thousands,NNS,O
1,0,of,IN,O
2,0,demonstrators,NNS,O
3,0,have,VBP,O
4,0,marched,VBN,O
5,0,through,IN,O
6,0,London,NNP,B-geo
7,0,to,TO,O
8,0,protest,VB,O
9,0,the,DT,O


In [16]:
texts_df.head(50)

,text,pos seq,tag seq
Sentence #,,,
0,Thousands of demonstrators have marched throug...,NNS IN NNS VBP VBN IN NNP TO VB DT NN IN NNP C...,O O O O O O B-geo O O O O O B-geo O O O O O B-...
1,Families of soldiers killed in the conflict jo...,NNS IN NNS VBN IN DT NN VBD DT NNS WP VBD NNS ...,O O O O O O O O O O O O O O O O O O B-per O O ...
2,They marched from the Houses of Parliament to ...,PRP VBD IN DT NNS IN NN TO DT NN IN NNP NNP .,O O O O O O O O O O O B-geo I-geo O
3,"Police put the number of marchers at 10,000 wh...",NNS VBD DT NN IN NNS IN CD IN NNS VBD PRP VBD ...,O O O O O O O O O O O O O O O
4,The protest comes on the eve of the annual con...,DT NN VBZ IN DT NN IN DT JJ NN IN NNP POS VBG ...,O O O O O O O O O O O B-geo O O B-org I-org O ...
5,The party is divided over Britain 's participa...,DT NN VBZ VBN IN NNP POS NN IN DT NNP NN CC DT...,O O O O O B-gpe O O O O B-geo O O O O O O O B-...
6,The London march came ahead of anti-war protes...,"DT NNP NN VBD RB IN JJ NNS NN IN JJ NNS , VBG ...",O B-geo O O O O O O O O O O O O B-geo O B-geo ...
7,The International Atomic Energy Agency is to h...,DT NNP NNP NNP NNP VBZ TO VB JJ NN IN NNS IN N...,O B-org I-org I-org I-org O O O O O O O O B-ge...
8,Iran this week restarted parts of the conversi...,NNP DT NN VBD NNS IN DT NN NN IN PRP$ NNP JJ NN .,B-gpe O O O O O O O O O O B-geo O O O


In [17]:
texts_df_rnn = data2.drop(['POS'], axis=1)
texts_df_rnn = texts_df_rnn.groupby('Sentence #').agg(list)
texts_df_rnn = texts_df_rnn.reset_index(drop=True)
texts_df_rnn.head()

,Word,Tag
0,"[Thousands, of, demonstrators, have, marched, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo..."
1,"[Families, of, soldiers, killed, in, the, conf...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"[They, marched, from, the, Houses, of, Parliam...","[O, O, O, O, O, O, O, O, O, O, O, B-geo, I-geo..."
3,"[Police, put, the, number, of, marchers, at, 1...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
4,"[The, protest, comes, on, the, eve, of, the, a...","[O, O, O, O, O, O, O, O, O, O, O, B-geo, O, O,..."


In [18]:
tag_list = []
for element in texts_df_rnn.Tag:
    for i in element:
        if i not in tag_list:
            tag_list.append(i)
num_classes = len(tag_list)
print('Number of classes : {} \nItems : {}'.format(num_classes,tag_list))

Number of classes : 17 
Items : ['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim', 'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve', 'I-eve', 'I-nat']


In [19]:
vocab = [str(item).lower() for element in texts_df_rnn.Word for item in element]
vocab = list(set(vocab))
nb_mots = len(vocab)
print('Number of different words : ',nb_mots)
vocab[:10]

Number of different words :  31814


['unappreciated',
 'sufis',
 'animist',
 'offstage',
 'gas-producing',
 'pre-trial',
 'retroactive',
 'aigle',
 'h.i.v.',
 'bogra']

In [20]:
import tensorflow as tf

tokenizer_txt = tf.keras.preprocessing.text.Tokenizer(num_words=nb_mots, filters=None)
tokenizer_txt.fit_on_texts(texts_df_rnn.Word)

word2idx = tokenizer_txt.word_index
idx2word = tokenizer_txt.index_word
vocab_size = tokenizer_txt.num_words

print(vocab_size)

31814


In [21]:
#show the 20 first words
for i in range(1,20):
    print(idx2word[i])

the
.
,
in
of
to
a
and
's
for
has
on
is
that
have
with
said
was
at


In [22]:
tokenizer_tag = tf.keras.preprocessing.text.Tokenizer(num_words=num_classes)
tokenizer_tag.fit_on_texts(texts_df_rnn.Tag)

word2idx_tag = tokenizer_tag.word_index
idx2word_tag = tokenizer_tag.index_word
vocab_size_tag = tokenizer_tag.num_words

word2idx_tag

{'o': 1,
 'b-geo': 2,
 'b-tim': 3,
 'b-org': 4,
 'i-per': 5,
 'b-per': 6,
 'i-org': 7,
 'b-gpe': 8,
 'i-geo': 9,
 'i-tim': 10,
 'b-art': 11,
 'b-eve': 12,
 'i-art': 13,
 'i-eve': 14,
 'b-nat': 15,
 'i-gpe': 16,
 'i-nat': 17}

In [23]:
X = tokenizer_txt.texts_to_sequences(texts_df_rnn.Word)
y = tokenizer_tag.texts_to_sequences(texts_df_rnn.Tag)
print(len(X[0]), X[0])
print(len(y[0]), y[0])

24 [253, 5, 966, 15, 1794, 237, 467, 6, 522, 1, 128, 4, 60, 8, 570, 1, 832, 5, 185, 89, 21, 14, 55, 2]
24 [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1]


In [24]:
longueur = texts_df_rnn.Tag.apply(lambda x: len(x))
long_max = max(longueur)
print("Tag max Tength : ",long_max)

X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=long_max, padding='post', truncating='post')
y = tf.keras.preprocessing.sequence.pad_sequences(y, maxlen=long_max, padding='post', truncating='post')

Tag max Tength :  104


In [25]:
print(len(X[0]), X[0])
print(len(y[0]), y[0])

104 [ 253    5  966   15 1794  237  467    6  522    1  128    4   60    8
  570    1  832    5  185   89   21   14   55    2    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]
104 [1 1 1 1 1 1 2 1 1 1 1 1 2 1 1 1 1 1 8 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [27]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, RNN, GRUCell, Dropout, Dense

out_dim = 64

model = Sequential(layers= [
    Embedding(input_dim=nb_mots+1, output_dim=out_dim, input_length=long_max),
    RNN(cell=GRUCell(out_dim), return_sequences=True),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 104, 64)           2036160   
                                                                 
 rnn (RNN)                   (None, 104, 64)           24960     
                                                                 
 dropout (Dropout)           (None, 104, 64)           0         
                                                                 
 dense (Dense)               (None, 104, 128)          8320      
                                                                 
 dropout_1 (Dropout)         (None, 104, 128)          0         
                                                                 
 dense_1 (Dense)             (None, 104, 17)           2193      
                                                                 
Total params: 2,071,633
Trainable params: 2,071,633
Non-

In [28]:
Sparse_loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

def Sparse_loss_function(real, pred):
    # Mask
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    # Avoid type error
    mask = tf.cast(mask, dtype=pred.dtype)
    # Loss function
    loss_ = Sparse_loss_object(real, pred)
    # Apply mask on loss function
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [ ]:
model.compile(optimizer='adam', loss=Sparse_loss_function, metrics=['accuracy'])
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5,batch_size=128,workers=-1)

Epoch 1/5
 49/300 [===>..........................] - ETA: 53s - loss: 0.2611 - accuracy: 0.7966